In [72]:
%matplotlib inline

import ee

ee.Initialize()

In [73]:
# HydroBASINS level 5
index = ee.FeatureCollection("users/gena/hybas_lev05_v1c_index")
basins = ee.FeatureCollection('ft:1IHRHUiWkgPXOzwNweeM89CzPYSfokjLlz7_0OTQl')

In [150]:
def get_upstream_catchments(basin_source):
    hybas_id = ee.Number(basin_source.get('HYBAS_ID'))
    upstream_ids = index.filter(ee.Filter.eq('hybas_id', hybas_id)).aggregate_array('parent_from')
    upstream_basins = basins.filter(ee.Filter.inList('hybas_id', upstream_ids)).merge(ee.FeatureCollection([basin_source]))
    
    return ee.Feature(upstream_basins.geometry().dissolve(450)).copyProperties(basin_source)
    
    return upstream_basins

In [75]:
# define query bounds (Prosser Creek Reservoir)
bounds = ee.Geometry.Polygon(
        [[[-120.17703806492915, 39.39640731015444],
          [-120.17695224301048, 39.36409996022785],
          [-120.12708745495274, 39.36409996023226],
          [-120.12751660052578, 39.39627465398903]]])

In [135]:
# define query bounds (Niger)
bounds = ee.Geometry.Polygon(
        [[[5.995833, 4.387513999999975], [7.704733999999998, 4.387513999999975],
          [7.704733999999998,7.925567000000025],[5.995833,7.925567000000025],
          [5.995833,4.387513999999975]]])

In [165]:
bounds = ee.Geometry.Point([6.70166015625, 7.079088026071719])

In [166]:
source_basins = basins.filterBounds(bounds)

# get parent catchments
upstream = source_basins.map(get_upstream_catchments)

In [167]:
print(ee.FeatureCollection(upstream.flatten()).aggregate_array('HYBAS_ID').getInfo())

[1050022420.0]


In [168]:
hybas_id = ee.Number(source_basins.first().get('HYBAS_ID'))
upstream_ids = index.filter(ee.Filter.eq('hybas_id', hybas_id)).aggregate_array('parent_from')
print(upstream_ids.getInfo())

upstream_basins = basins.filter(ee.Filter.inList('HYBAS_ID', upstream_ids)).merge(ee.FeatureCollection([source_basins.first()]))
print(upstream_basins.size().getInfo())

[1050683010, 1050635010, 1050741380, 1050694020, 1050641670, 1050888070, 1050874630, 1050839560, 1050742280, 1050722570, 1050635020, 1050870030, 1050736270, 1050874640, 1050682900, 1050669460, 1050827030, 1050736280, 1050870170, 1050669470, 1050722720, 1050827040, 1050680480, 1050668450, 1050887720, 1050761260, 1050515500, 1050741300, 1050761270, 1050689850, 1050515770, 1050677950, 1050892990, 1050513090, 1050909890, 1050846020, 1050677960, 1050893000, 1050742220, 1050909900, 1050715340, 1050737100, 1050513230, 1050747470, 1050652110, 1050690130, 1050696660, 1050577750, 1050737110, 1050760280, 1050668760, 1050715100, 1050577760, 1050710240, 1050760290, 1050603880, 1050755820, 1050710380, 1050908780, 1050680300, 1050514030, 1050839280, 1050909040, 1050694130, 1050603890, 1050513910, 1050755960, 1050845560, 1050747640, 1050641660, 1050651900, 1050696830]
73
